In [1]:
from datetime import datetime, timedelta
import gym
from gym.utils import seeding
import numpy as np
import random
import pandas as pd
from gym import spaces

class Gridworld_v1(gym.Env): # define custom environment as subclass of gym.Env
    # GLOBAL VARIABLES:

    POS_MIN = 0
    POS_MAX = 100
    MAX_STEPS = 200
    REWARD_AWAY = -1
    REWARD_GOAL = MAX_STEPS

    metadata = {
     "render.modes": ["human"]
   }
    
    def __init__(self,env_config = None):
        env_config = env_config or {}
        num_states = 100
        self.action_space = gym.spaces.Discrete(4) # 4 valid actions (1- up, 2-down, 3-left, 4-right)
        self.observation_space = gym.spaces.Discrete(num_states)
        self.final_hub = 43
        #self.hubs = [(random.randrange(0, 9), random.randrange(0, 9)) for i in range(5)] 

        #if self.final_hub in self.hubs:
        #    self.hubs.remove(self.final_hub)

        self.seed()
        self.reset()
        random_number = np.random.randint(31536000) # random seconds number in order to generate a random date
        self.time=datetime(2021,1,1,12,0,0)+timedelta(seconds=random_number)
   
    
    def step(self, action):
        assert self.action_space.contains(action)
        nxtposition=0
        # if done:
        # #     should never reach this point
        #      print("EPISODE DONE!!!")
        # elif self.count == self.MAX_STEPS:
        #         done = True
        # else:
            #     assert self.action_space.contains(action)
        self.count += 1

        if action == 'u' or action == 1: #up
                nxtposition = self.state - 10
        elif action == 'd' or action == 2: #down
                nxtposition = self.state + 1
        elif action == 'l' or action == 3: #left
                nxtposition = self.state-1
        else: #right
                nxtposition = self.state+ 1
       
        done = self.count >= self.MAX_STEPS
            
        if (nxtposition> 0) and (nxtposition <= (self.POS_MAX)):
                    self.state = nxtposition
                    self.time=self.time+timedelta(minutes=1)
        
        else:
                reward = self.REWARD_AWAY
        if self.state == self.final_hub:
            # on goal now
            reward = self.REWARD_GOAL
            done = True
        else: 
            # moving away from goal
            reward = self.REWARD_AWAY
        
        return self.state, reward, done, self.info
      
    def availableActions(self):
        position=str(self.position)
        start_timestamp=str(self.time)
        list=[]
        time_window=5
        end_timestamp = str(start_timestamp + timedelta(minutes=time_window))
        grid=pd.read_csv('rl\env\data_gridworld_timestamps.csv')
        #grid=grid.head(10)
        paths=grid['Path Timestamp']
        for index in range(len(paths)):
            #print(grid['Path Timestamp'][index])
            dict = eval(grid['Path Timestamp'][index])
            for tupel_position in dict:
                #print(tupel_position ,dict[tupel_position])
                position_timestamp=dict[tupel_position]
                if str(tupel_position) == position and start_timestamp <= position_timestamp \
                and end_timestamp >= dict[tupel_position] and str(tupel_position) != grid['Dropoff Coordinates'][index]:
                   list.append([position_timestamp,grid['Dropoff Coordinates'][index]])
                   # TODO slice and retrun dictionary in order to get only the route to the final node from the current node
        return list



    def render(self, mode="human"): # method for visualization; optional
        #s = "position: {:2d}  reward: {:2d} "
        # print(s.format(self.position, self.reward))
        
        # print("Position: "+ str(self.position)+ " Reward: "+ str(self.reward)+ " Time: "+str(self.time))
        pass


    def reset(self):
        self.count = 0
        #self.state = random.randint(1, 100)
        self.state = 42
        self.info = {}
        return self.state
        

    def seed(self, seed=None): # optional
        # self.np_random, seed = seeding.np_random(seed)
        # print(self.np_random)
        # return [seed]
        pass

    def close(self): # optional
        pass


In [2]:
#without ppo trainer
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(env.MAX_STEPS):
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print(sum_reward)
            break
    return sum_reward
for i in range(20):
    run_one_episode (Gridworld_v1())

-200
200
-200
-200
200
-200
200
200
-200
200
-200
200
200
-200
200
200
-200
-200
200
-200


In [3]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [4]:
ray.init(ignore_reinit_error=True)

2022-03-21 22:56:52,133	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-03-21_22-56-50_290786_48433/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-21_22-56-50_290786_48433/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-21_22-56-50_290786_48433',
 'metrics_export_port': 61172,
 'gcs_address': '127.0.0.1:58907',
 'node_id': '844458327e050b9b862cb26cdde970d41106badad0a6c58679560783'}

In [5]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10

In [6]:
trainer = PPOTrainer(trainer_config, Gridworld_v1)

2022-03-21 22:56:53,645	INFO trainer.py:2054 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-03-21 22:56:53,648	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-03-21 22:56:53,649	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=48477) 2022-03-21 22:56:56,409	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-03-21 22:56:56,739	WARNING util.py:55 -- Install gputil for GPU system mo

In [7]:
checkpoint_root = "tmp/ppo/gridworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [8]:
results = []
episode_data = []
episode_json = []

for n in range(20):
    result = trainer.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

2022-03-21 22:56:56,987	WARNING deprecation.py:45 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!
2022-03-21 22:56:57,227	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


  1: Min/Mean/Max reward: -200.0000/-200.0000/-200.0000, len mean: 200.0000. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000001/checkpoint-1
  2: Min/Mean/Max reward: -200.0000/ 49.5000/200.0000, len mean:  76.1250. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000002/checkpoint-2
  3: Min/Mean/Max reward: -200.0000/ 17.8182/200.0000, len mean:  91.8182. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000003/checkpoint-3
  4: Min/Mean/Max reward: -200.0000/ 12.4000/200.0000, len mean:  94.8000. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000004/checkpoint-4
  5: Min/Mean/Max reward: -200.0000/ -0.7778/200.0000, len mean: 101.2778. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000005/checkpoint-5
  6: Min/Mean/Max reward: -200.0000/ 15.8333/200.0000, len mean:  93.0417. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000006/checkpoint-6
  7: Min/Mean/Max reward: -200.0000/ -0.7692/200.0000, len mean: 101.2692. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000007/che